In [1]:
import tensorly as tl
import numpy as np
from numpy import linalg as la
from sympy import *

from tensorly.decomposition import parafac
from tensorly.decomposition import non_negative_parafac_hals

In [26]:
def check_rank(tensor, rank, non_neg=True, n=10, tol=0.001, p=False):
    if non_neg:
        for k in range(n):
            weights, factors = non_negative_parafac_hals(tensor, n_iter_max=1000000, rank=rank, init='random')
            full = tl.cp_to_tensor((weights, factors))
            diff = (full - tensor) / tensor
            if p:
                # print(full)
                print(tl.max(abs(diff)))
            if tl.max(abs(diff)) < tol:
                return True
    else:
        for k in range(n):
            weights, factors = parafac(tensor, n_iter_max=1000000, rank=rank)
            full = tl.cp_to_tensor((weights, factors))
            diff = (full - tensor) / tensor
            if p:
                print(tl.max(abs(diff)))
            if tl.max(abs(diff)) < tol:
                return True
    return False

#generates random rank 3 tensors
def rank_tree():
    return (low_tensor() + low_tensor() + low_tensor()) 

#generates random rank 4+ (hopefully) tensors
def rank_for():
    return  (low_tensor() + low_tensor() + low_tensor() + low_tensor())


# generates rank 1, 2x2x3 tensors
def low_tensor():
    max = 40
    a = np.random.randint(1, max, size=4)
    b = np.random.randint(1, max, size=4)
    c = np.random.randint(1, max, size=4)
    tens = tl.tensor(np.kron(np.kron(a, b), c).reshape(4, 4, 4)) * 0.1
    return (tens,a,b,c)

In [27]:
def check_for(p):
    d1 = Matrix(p[0])
    d2 = Matrix(p[1])
    d3 = Matrix(p[2])
    d2a = d2.adjugate()
    a1 = d1 * d2a * d3 - d3* d2a * d1
    #print(a1)
    return check(a1)

def check(a):
  #account for numerical errors
    return max(abs(a)) < 5

In [28]:
res = {True:0, False:0}
wrong = []
for i in range(1000000):
    t1 = low_tensor()
    t2 = low_tensor()
    t3 = low_tensor()
    t4 = low_tensor()
    t5 = low_tensor()
    v = np.random.randint(0,100)
    t = t1[0] + t2[0] + t3[0] + t4[0] + t5[0]
    if v%2 == 0:
        t = np.random.randint(0,100000,size = (3,3,3))
    key = check_for(t)
    res[key] = res[key] + 1
    if key:
        #print(t1, t2, t3, t4, t5, t, v)
        #print(check_rank(t,3, tol = 0.01))
        wrong.append((t1,t2,t3,t4,t5,t,v))
print(res)

{True: 0, False: 1000000}


In [25]:
print(wrong)

[((array([[[117. ,  97.5,  65. , 110.5],
        [135. , 112.5,  75. , 127.5],
        [  9. ,   7.5,   5. ,   8.5],
        [135. , 112.5,  75. , 127.5]],

       [[163.8, 136.5,  91. , 154.7],
        [189. , 157.5, 105. , 178.5],
        [ 12.6,  10.5,   7. ,  11.9],
        [189. , 157.5, 105. , 178.5]],

       [[ 93.6,  78. ,  52. ,  88.4],
        [108. ,  90. ,  60. , 102. ],
        [  7.2,   6. ,   4. ,   6.8],
        [108. ,  90. ,  60. , 102. ]],

       [[444.6, 370.5, 247. , 419.9],
        [513. , 427.5, 285. , 484.5],
        [ 34.2,  28.5,  19. ,  32.3],
        [513. , 427.5, 285. , 484.5]]]), array([ 5,  7,  4, 19]), array([13, 15,  1, 15]), array([18, 15, 10, 17])), (array([[[ 41.6,  44.8,  19.2,  25.6],
        [124.8, 134.4,  57.6,  76.8],
        [124.8, 134.4,  57.6,  76.8],
        [124.8, 134.4,  57.6,  76.8]],

       [[ 93.6, 100.8,  43.2,  57.6],
        [280.8, 302.4, 129.6, 172.8],
        [280.8, 302.4, 129.6, 172.8],
        [280.8, 302.4, 129.6, 172.8